In [3]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta

# =====================================================
# CONFIGURAÇÕES
# =====================================================
np.random.seed(42)
random.seed(42)

TOTAL_OPERADORES = 120
TURNOS = ["Manhã", "Tarde", "Noite"]

LINHAS_PRODUCAO = [
    "L-B01 | Base Pesada – Soldadura Automática",
    "L-M02 | Base/Meio – Processo Tradicional",
    "L-M03 | Meio – Produção Padronizada",
    "L-T04 | Topo – Alta Precisão",
    "L-N05 | Linha Nova – Ramp-up"
]

SECAO_POR_LINHA = {
    "L-B01 | Base Pesada – Soldadura Automática": "Base",
    "L-M02 | Base/Meio – Processo Tradicional": "Base/Meio",
    "L-M03 | Meio – Produção Padronizada": "Meio",
    "L-T04 | Topo – Alta Precisão": "Topo",
    "L-N05 | Linha Nova – Ramp-up": "Meio"
}

# =====================================================
# CRIAR OPERADORES (ID ÚNICO + NOME ÚNICO)
# =====================================================
nomes_base = [
    "João", "Pedro", "Miguel", "Carlos", "Rui", "Tiago", "Bruno",
    "André", "Paulo", "Daniel", "Sérgio", "Ricardo", "Nuno"
]
apelidos = [
    "Silva", "Santos", "Ferreira", "Pereira", "Costa",
    "Oliveira", "Rodrigues", "Martins"
]

operadores = []

for i in range(1, TOTAL_OPERADORES + 1):
    linha = random.choice(LINHAS_PRODUCAO)
    nome_unico = f"{random.choice(nomes_base)} {random.choice(apelidos)} ({i:03d})"
    operadores.append({
        "operador_id": f"OPR-{i:03d}",
        "nome_operador": nome_unico,
        "linha_producao": linha,
        "secao": SECAO_POR_LINHA[linha],
        "experiencia": random.choice(["Junior", "Pleno", "Senior"])
    })

df_operadores = pd.DataFrame(operadores)

# =====================================================
# CARREGAR ORDENS DE PRODUÇÃO
# =====================================================
df_producao = pd.read_excel("producao_ordens.xlsx")

# =====================================================
# GERAR TURNOS
# =====================================================
registos = []

for _, ordem in df_producao.iterrows():

    # operadores da linha correta
    operadores_linha = df_operadores[df_operadores["linha_producao"] == ordem["linha_producao"]].to_dict('records')

    num_operadores = random.randint(2, 6)
    operadores_usados = random.sample(operadores_linha, min(num_operadores, len(operadores_linha)))

    data_atual = pd.to_datetime(ordem["data_inicio_real"])
    data_fim = pd.to_datetime(ordem["data_fim_real"])

    while data_atual <= data_fim:
        # incluir sábado aleatório
        if data_atual.weekday() < 6:  
            for op in operadores_usados:

                turno = random.choices(TURNOS, weights=[0.45, 0.35, 0.20])[0]

                horas_base = random.uniform(6.5, 8.0)
                if turno == "Noite":
                    horas_extra = random.uniform(1.0, 3.0)
                else:
                    horas_extra = random.uniform(0.0, 1.5)

                if op["experiencia"] == "Senior":
                    horas_extra *= 0.7

                registos.append({
                    "operador_id": op["operador_id"],
                    "nome_operador": op["nome_operador"],
                    "secao": op["secao"],
                    "linha_producao": op["linha_producao"],
                    "turno": turno,
                    "data": data_atual.date(),
                    "horas_trabalhadas": round(horas_base, 2),
                    "horas_extra": round(horas_extra, 2),
                    "ordem_id": ordem["ordem_id"],
                    "ano": ordem["ano"],
                    "mes": ordem["mes"]
                })

        data_atual += timedelta(days=1)

# =====================================================
# DATAFRAME FINAL
# =====================================================
df_turnos = pd.DataFrame(registos)
df_turnos = df_turnos.sort_values(by=["ano", "data", "linha_producao"]).reset_index(drop=True)

# =====================================================
# EXPORTAÇÃO
# =====================================================
df_turnos.to_excel("turnos_operadores.xlsx", index=False)

print("✔ turnos_operadores.xlsx criado com sucesso!")
print(df_turnos.head())
print(f"Total de linhas: {len(df_turnos)}")


✔ turnos_operadores.xlsx criado com sucesso!
  operador_id        nome_operador secao                linha_producao  turno  \
0     OPR-014  Tiago Pereira (014)  Meio  L-N05 | Linha Nova – Ramp-up  Manhã   
1     OPR-087   Carlos Costa (087)  Meio  L-N05 | Linha Nova – Ramp-up  Tarde   
2     OPR-049   Nuno Pereira (049)  Meio  L-N05 | Linha Nova – Ramp-up  Tarde   
3     OPR-081    João Santos (081)  Meio  L-N05 | Linha Nova – Ramp-up  Manhã   
4     OPR-119   Carlos Costa (119)  Meio  L-N05 | Linha Nova – Ramp-up  Tarde   

         data  horas_trabalhadas  horas_extra  ordem_id   ano      mes  
0  2021-12-31               7.85         0.53  OP-00654  2022  January  
1  2021-12-31               6.99         1.05  OP-00654  2022  January  
2  2021-12-31               6.79         0.70  OP-00654  2022  January  
3  2021-12-31               7.62         0.26  OP-00654  2022  January  
4  2021-12-31               7.11         1.25  OP-00654  2022  January  
Total de linhas: 51496


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# =====================================================
# CONFIGURAÇÕES GERAIS
# =====================================================
np.random.seed(42)
random.seed(42)

TOTAL_ORDENS = 1000
ANOS = [2022, 2023, 2024]

HORAS_POR_DIA = 8

# =====================================================
# DEFINIÇÃO DAS LINHAS DE PRODUÇÃO
# =====================================================
LINHAS_PRODUCAO = {
    "L-B01 | Base Pesada – Soldadura Automática": {
        "secao": "Base",
        "duracao_media_dias": 18,
        "variacao_dias": 6,
        "prob_atraso": 0.55
    },
    "L-M02 | Base/Meio – Processo Tradicional": {
        "secao": "Base/Meio",
        "duracao_media_dias": 14,
        "variacao_dias": 5,
        "prob_atraso": 0.45
    },
    "L-M03 | Meio – Produção Padronizada": {
        "secao": "Meio",
        "duracao_media_dias": 10,
        "variacao_dias": 4,
        "prob_atraso": 0.35
    },
    "L-T04 | Topo – Alta Precisão": {
        "secao": "Topo",
        "duracao_media_dias": 8,
        "variacao_dias": 3,
        "prob_atraso": 0.25
    },
    "L-N05 | Linha Nova – Ramp-up": {
        "secao": "Meio",
        "duracao_media_dias": 16,
        "variacao_dias": 7,
        "prob_atraso": 0.65
    }
}

# =====================================================
# FUNÇÕES AUXILIARES SEGURAS
# =====================================================
def gerar_data_inicio(ano):
    """Gera uma data válida e realista"""
    mes = random.randint(1, 12)
    dia = random.randint(1, 20)  # evita fins de mês problemáticos
    return datetime(ano, mes, dia)

def gerar_duracao(media, variacao):
    dur = int(np.random.normal(media, variacao))
    return max(3, min(dur, 25))  # limites industriais

def gerar_atraso(prob_atraso):
    if random.random() < prob_atraso:
        return random.randint(1, 10)     # atraso
    else:
        return random.randint(-5, 2)     # adiantado ou no prazo

# =====================================================
# GERAÇÃO DOS DADOS
# =====================================================
registos = []

for i in range(1, TOTAL_ORDENS + 1):

    linha = random.choice(list(LINHAS_PRODUCAO.keys()))
    config = LINHAS_PRODUCAO[linha]

    ano = random.choice(ANOS)

    data_inicio_planeada = gerar_data_inicio(ano)

    duracao_planeada = gerar_duracao(
        config["duracao_media_dias"],
        config["variacao_dias"]
    )

    data_fim_planeada = data_inicio_planeada + timedelta(days=duracao_planeada)

    atraso_dias = gerar_atraso(config["prob_atraso"])

    data_inicio_real = data_inicio_planeada + timedelta(days=random.randint(-1, 2))
    data_fim_real = data_fim_planeada + timedelta(days=atraso_dias)

    horas_planeadas = duracao_planeada * HORAS_POR_DIA

    horas_reais = int(
        horas_planeadas * random.uniform(0.9, 1.1) +
        max(0, atraso_dias) * HORAS_POR_DIA * random.uniform(0.5, 1.2)
    )

    horas_reais = max(int(horas_planeadas * 0.8), horas_reais)

    if data_fim_real > data_fim_planeada:
        estado = "Atrasada"
    elif data_fim_real < data_fim_planeada:
        estado = "Adiantada"
    else:
        estado = "No prazo"

    registos.append({
        "ordem_id": f"OP-{i:05d}",
        "linha_producao": linha,
        "tipo_secao": config["secao"],
        "data_inicio_planeada": data_inicio_planeada.date(),
        "data_fim_planeada": data_fim_planeada.date(),
        "data_inicio_real": data_inicio_real.date(),
        "data_fim_real": data_fim_real.date(),
        "horas_planeadas": horas_planeadas,
        "horas_reais": horas_reais,
        "quantidade_planeada": random.randint(1, 4),
        "quantidade_produzida": random.randint(1, 4),
        "estado_ordem": estado,
        "ano": ano,
        "mes": data_inicio_planeada.strftime("%B")
    })

# =====================================================
# DATAFRAME FINAL
# =====================================================
df_producao = pd.DataFrame(registos)

# Ordenação ajuda muito no Power BI
df_producao = df_producao.sort_values(
    by=["ano", "data_inicio_planeada"]
).reset_index(drop=True)

# =====================================================
# EXPORTAÇÃO
# =====================================================
df_producao.to_excel("producao_ordens.xlsx", index=False)

print("✔ producao_ordens.xlsx criado com sucesso")
print(df_producao.head())


In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# =====================================================
# CONFIGURAÇÕES
# =====================================================
np.random.seed(42)
random.seed(42)

TOTAL_ENTRADAS = 1500
ANOS = [2022, 2023, 2024]

# =====================================================
# FORNECEDORES
# =====================================================
fornecedores = [
    {"id": "F-01", "nome": "Arcelor Steel Iberia", "fiabilidade": 0.85},
    {"id": "F-02", "nome": "SSAB Europe", "fiabilidade": 0.90},
    {"id": "F-03", "nome": "Aço Norte", "fiabilidade": 0.70},
    {"id": "F-04", "nome": "Metalúrgica Atlântica", "fiabilidade": 0.65},
    {"id": "F-05", "nome": "SteelWorks Global", "fiabilidade": 0.80},
    {"id": "F-06", "nome": "Fornecedor Local", "fiabilidade": 0.60}
]

# =====================================================
# MATERIAIS
# =====================================================
materiais = [
    {"material": "Chapa Grossa", "tipo": "Base", "custo_ton": (850, 1100)},
    {"material": "Chapa Média", "tipo": "Meio", "custo_ton": (700, 900)},
    {"material": "Chapa Fina", "tipo": "Topo", "custo_ton": (600, 800)},
    {"material": "Flanges", "tipo": "Estrutural", "custo_ton": (900, 1200)},
    {"material": "Parafusos Industriais", "tipo": "Fixação", "custo_ton": (1200, 1600)}
]

armazens = ["Aveiro Norte", "Aveiro Sul", "Parque Externo"]

# =====================================================
# FUNÇÕES SEGURAS
# =====================================================
def gerar_data(ano):
    mes = random.randint(1, 12)
    dia = random.randint(1, 20)
    return datetime(ano, mes, dia)

def gerar_atraso(fiabilidade):
    if random.random() < fiabilidade:
        return random.randint(-3, 3)
    else:
        return random.randint(4, 12)

# =====================================================
# GERAÇÃO DOS DADOS
# =====================================================
registos = []

for i in range(1, TOTAL_ENTRADAS + 1):

    ano = random.choice(ANOS)
    fornecedor = random.choice(fornecedores)
    material = random.choice(materiais)

    data_prevista = gerar_data(ano)
    atraso = gerar_atraso(fornecedor["fiabilidade"])
    data_real = data_prevista + timedelta(days=atraso)

    peso = round(random.uniform(5, 40), 2)
    quantidade = random.randint(1, 5)

    custo_por_ton = round(
        random.uniform(material["custo_ton"][0], material["custo_ton"][1]), 2
    )

    custo_total = round(peso * custo_por_ton, 2)

    registos.append({
        "entrada_id": f"ENT-{i:05d}",
        "fornecedor_id": fornecedor["id"],
        "fornecedor_nome": fornecedor["nome"],
        "material": material["material"],
        "tipo_material": material["tipo"],
        "peso_toneladas": peso,
        "quantidade": quantidade,
        "custo_por_ton": custo_por_ton,
        "custo_total": custo_total,
        "data_prevista_entrega": data_prevista.date(),
        "data_real_entrega": data_real.date(),
        "armazem": random.choice(armazens),
        "ano": ano,
        "mes": data_prevista.strftime("%B")
    })

# =====================================================
# DATAFRAME FINAL
# =====================================================
df_entradas = pd.DataFrame(registos)

df_entradas = df_entradas.sort_values(
    by=["ano", "data_prevista_entrega"]
).reset_index(drop=True)

# =====================================================
# EXPORTAÇÃO
# =====================================================
df_entradas.to_excel("entradas_materia_prima.xlsx", index=False)

print("✔ entradas_materia_prima.xlsx criado com sucesso")
print(df_entradas.head())
print(f"Total de linhas: {len(df_entradas)}")


In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta

# =====================================================
# CONFIGURAÇÕES
# =====================================================
np.random.seed(42)
random.seed(42)

# =====================================================
# TRANSPORTADORAS
# =====================================================
transportadoras = [
    {"nome": "TransWind Logistics", "fiabilidade": 0.90, "custo_km": 2.8},
    {"nome": "Europa Heavy Transport", "fiabilidade": 0.85, "custo_km": 2.5},
    {"nome": "Atlântico Cargas", "fiabilidade": 0.70, "custo_km": 2.1},
    {"nome": "Carga Rápida", "fiabilidade": 0.65, "custo_km": 1.9}
]

destinos = [
    {"pais": "Portugal", "km": 300},
    {"pais": "Espanha", "km": 700},
    {"pais": "França", "km": 1200},
    {"pais": "Alemanha", "km": 2200},
    {"pais": "Dinamarca", "km": 2600}
]

# =====================================================
# CARREGAR PRODUÇÃO
# =====================================================
df_producao = pd.read_excel("producao_ordens.xlsx")

# =====================================================
# FUNÇÕES
# =====================================================
def gerar_atraso(fiabilidade):
    if random.random() < fiabilidade:
        return random.randint(-3, 2)
    else:
        return random.randint(3, 7)

def peso_por_linha(linha):
    if "Base" in linha:
        return random.uniform(180, 320)
    if "Meio" in linha:
        return random.uniform(90, 160)
    return random.uniform(40, 80)

# =====================================================
# GERAÇÃO DOS DADOS
# =====================================================
registos = []

for i, ordem in df_producao.iterrows():

    transportadora = random.choice(transportadoras)
    destino = random.choice(destinos)

    data_prevista = pd.to_datetime(ordem["data_fim_real"]) + timedelta(days=random.randint(1, 4))
    atraso = gerar_atraso(transportadora["fiabilidade"])
    data_real = data_prevista + timedelta(days=atraso)

    peso = round(peso_por_linha(ordem["linha_producao"]), 2)

    custo = round(
        peso * destino["km"] * transportadora["custo_km"] / 1000,
        2
    )

    if data_real > data_prevista:
        estado = "Atrasada"
    elif data_real < data_prevista:
        estado = "Adiantada"
    else:
        estado = "No prazo"

    registos.append({
        "expedicao_id": f"EXP-{i+1:05d}",
        "ordem_id": ordem["ordem_id"],
        "linha_producao": ordem["linha_producao"],
        "transportadora": transportadora["nome"],
        "destino": destino["pais"],
        "peso_total_ton": peso,
        "data_saida_prevista": data_prevista.date(),
        "data_saida_real": data_real.date(),
        "custo_transporte": custo,
        "estado_expedicao": estado,
        "ano": ordem["ano"],
        "mes": ordem["mes"]
    })

# =====================================================
# DATAFRAME FINAL
# =====================================================
df_expedicoes = pd.DataFrame(registos)

df_expedicoes = df_expedicoes.sort_values(
    by=["ano", "data_saida_prevista"]
).reset_index(drop=True)

# =====================================================
# EXPORTAÇÃO
# =====================================================
df_expedicoes.to_excel("expedicoes_torres.xlsx", index=False)

print("✔ expedicoes_torres.xlsx criado com sucesso")
print(df_expedicoes.head())
print(f"Total de linhas: {len(df_expedicoes)}")


In [ ]:
import pandas as pd
import numpy as np
import random

# =====================================================
# CONFIGURAÇÕES
# =====================================================
np.random.seed(42)
random.seed(42)

# =====================================================
# CARREGAR FICHEIROS BASE
# =====================================================
df_prod = pd.read_excel("producao_ordens.xlsx")
df_turnos = pd.read_excel("turnos_operadores.xlsx")
df_exped = pd.read_excel("expedicoes_torres.xlsx")

# =====================================================
# PARÂMETROS DE CUSTO
# =====================================================
CUSTO_HORA_BASE = 22
CUSTO_HORA_EXTRA = 32

CUSTO_MAQUINA_LINHA = {
    "L-B01 | Base Pesada – Soldadura Automática": 85,
    "L-M02 | Base/Meio – Processo Tradicional": 55,
    "L-M03 | Meio – Produção Padronizada": 45,
    "L-T04 | Topo – Alta Precisão": 50,
    "L-N05 | Linha Nova – Ramp-up": 70
}

# =====================================================
# CUSTO DE MÃO DE OBRA REAL
# =====================================================
mao_obra = (
    df_turnos
    .groupby("ordem_id")
    .agg({
        "horas_trabalhadas": "sum",
        "horas_extra": "sum"
    })
    .reset_index()
)

mao_obra["custo_mao_obra"] = (
    mao_obra["horas_trabalhadas"] * CUSTO_HORA_BASE +
    mao_obra["horas_extra"] * CUSTO_HORA_EXTRA
)

# =====================================================
# CUSTO LOGÍSTICO REAL
# =====================================================
logistica = (
    df_exped
    .groupby("ordem_id")["custo_transporte"]
    .sum()
    .reset_index()
    .rename(columns={"custo_transporte": "custo_logistico"})
)

# =====================================================
# CONSTRUÇÃO DO FICHEIRO FINAL
# =====================================================
registos = []

for _, ordem in df_prod.iterrows():

    ordem_id = ordem["ordem_id"]
    linha = ordem["linha_producao"]

    horas_planeadas = ordem["horas_planeadas"]
    horas_reais = ordem["horas_reais"]

    # ---------- MATERIAL ----------
    custo_material_planeado = horas_planeadas * random.uniform(45, 65)
    custo_material_real = custo_material_planeado * random.uniform(0.95, 1.15)

    # ---------- MÃO DE OBRA ----------
    custo_mo_real = mao_obra.loc[
        mao_obra["ordem_id"] == ordem_id,
        "custo_mao_obra"
    ].values

    custo_mo_real = float(custo_mo_real[0]) if len(custo_mo_real) > 0 else 0

    custo_mo_planeado = custo_mo_real * random.uniform(0.85, 0.95)

    # ---------- MÁQUINA ----------
    custo_maquina_planeado = horas_planeadas * CUSTO_MAQUINA_LINHA[linha]
    custo_maquina_real = horas_reais * CUSTO_MAQUINA_LINHA[linha] * random.uniform(1.0, 1.1)

    # ---------- LOGÍSTICA ----------
    custo_log = logistica.loc[
        logistica["ordem_id"] == ordem_id,
        "custo_logistico"
    ].values

    custo_log = float(custo_log[0]) if len(custo_log) > 0 else 0

    # ---------- TOTAIS ----------
    total_planeado = (
        custo_material_planeado +
        custo_mo_planeado +
        custo_maquina_planeado
    )

    total_real = (
        custo_material_real +
        custo_mo_real +
        custo_maquina_real +
        custo_log
    )

    registos.append({
        "ordem_id": ordem_id,
        "linha_producao": linha,
        "custo_material": round(custo_material_real, 2),
        "custo_mao_obra": round(custo_mo_real, 2),
        "custo_maquina": round(custo_maquina_real, 2),
        "custo_logistico": round(custo_log, 2),
        "custo_total_planeado": round(total_planeado, 2),
        "custo_total_real": round(total_real, 2),
        "desvio_custo": round(total_real - total_planeado, 2),
        "ano": ordem["ano"],
        "mes": ordem["mes"]
    })

# =====================================================
# DATAFRAME FINAL
# =====================================================
df_custos = pd.DataFrame(registos)

df_custos = df_custos.sort_values(
    by=["ano", "ordem_id"]
).reset_index(drop=True)

# =====================================================
# EXPORTAÇÃO
# =====================================================
df_custos.to_excel("custos_producao.xlsx", index=False)

print("✔ custos_producao.xlsx criado com sucesso")
print(df_custos.head())
print(f"Total de linhas: {len(df_custos)}")


In [ ]:
import pandas as pd
import numpy as np
import random

# =====================================================
# CONFIGURAÇÕES
# =====================================================
np.random.seed(42)
random.seed(42)

# =====================================================
# CARREGAR PRODUÇÃO
# =====================================================
df_prod = pd.read_excel("producao_ordens.xlsx")

# =====================================================
# PARÂMETROS DE QUALIDADE POR LINHA
# =====================================================
prob_nc_linha = {
    "L-B01 | Base Pesada – Soldadura Automática": 0.45,
    "L-M02 | Base/Meio – Processo Tradicional": 0.35,
    "L-M03 | Meio – Produção Padronizada": 0.20,
    "L-T04 | Topo – Alta Precisão": 0.15,
    "L-N05 | Linha Nova – Ramp-up": 0.50
}

tipos_defeito = [
    "Soldadura",
    "Dimensional",
    "Acabamento",
    "Material",
    "Montagem"
]

gravidades = {
    "Baixa": {"horas": (2, 5), "custo_hora": 35},
    "Média": {"horas": (6, 12), "custo_hora": 45},
    "Alta": {"horas": (15, 30), "custo_hora": 60}
}

# =====================================================
# GERAÇÃO DOS DADOS
# =====================================================
registos = []
contador_nc = 1

for _, ordem in df_prod.iterrows():

    linha = ordem["linha_producao"]
    secao = ordem["tipo_secao"]

    if random.random() < prob_nc_linha[linha]:

        num_nc = random.randint(1, 3)

        for _ in range(num_nc):

            gravidade = random.choices(
                list(gravidades.keys()),
                weights=[0.45, 0.35, 0.20],
                k=1
            )[0]

            horas = random.randint(
                gravidades[gravidade]["horas"][0],
                gravidades[gravidade]["horas"][1]
            )

            custo = horas * gravidades[gravidade]["custo_hora"] * random.uniform(1.0, 1.2)

            registos.append({
                "nc_id": f"NC-{contador_nc:05d}",
                "ordem_id": ordem["ordem_id"],
                "linha_producao": linha,
                "secao": secao,
                "tipo_defeito": random.choice(tipos_defeito),
                "gravidade": gravidade,
                "horas_retrabalho": horas,
                "custo_retrabalho": round(custo, 2),
                "data_nc": ordem["data_fim_real"],
                "ano": ordem["ano"],
                "mes": ordem["mes"]
            })

            contador_nc += 1

# =====================================================
# DATAFRAME FINAL
# =====================================================
df_nc = pd.DataFrame(registos)

df_nc = df_nc.sort_values(
    by=["ano", "linha_producao"]
).reset_index(drop=True)

# =====================================================
# EXPORTAÇÃO
# =====================================================
df_nc.to_excel("nao_conformidades.xlsx", index=False)

print("✔ nao_conformidades.xlsx criado com sucesso")
print(df_nc.head())
print(f"Total de NCs: {len(df_nc)}")
